# **Lab 8**
- Juan Pablo Solis
- Brandon Reyes
- Carlos Valladares



### **Task 1 Teoria**

#### ***Investigar el algoritmo AC-3 y su relación con el algoritmo de backtracking search***

El algoritmo AC-3 (Arc Consistency 3) se usa para reducir los dominios de las variables en problemas de satisfacción de restricciones (CSP), eliminando valores que no pueden formar parte de ninguna solución válida. Su relación con el algoritmo de backtracking search es que AC-3 puede aplicarse como un paso previo o durante el proceso de búsqueda para reducir el espacio de búsqueda, haciendo que el backtracking sea más eficiente al detectar inconsistencias antes de tomar decisiones.

#### ***Defina el termino "Arc Consistency"***
La arc consistency significa que, para cada valor posible de una variable, debe haber al menos un valor compatible en la variable relacionada (según las restricciones del problema). Así se asegura que no haya valores que lleven a un conflicto inmediato


### **Algoritmo Backtracking**

In [1]:
import time

# Datos del problema
examenes = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7']
dias = ['Lunes', 'Martes', 'Miércoles']
estudiantes = {
    'A': ['E1', 'E2', 'E3'],
    'B': ['E2', 'E4', 'E5'],
    'C': ['E1', 'E4', 'E6'],
    'D': ['E3', 'E5', 'E7']
}

# Verifica si una asignación es válida
def es_valida(asignacion):
    examenes_por_dia = {dia: [] for dia in dias}
    for examen, dia in asignacion.items():
        examenes_por_dia[dia].append(examen)

    # Verificar que ningún estudiante tenga más de un examen por día
    for estudiante, cursos in estudiantes.items():
        examenes_estudiante = {dia: 0 for dia in dias}
        for curso in cursos:
            if curso in asignacion:
                dia = asignacion[curso]
                examenes_estudiante[dia] += 1
                if examenes_estudiante[dia] > 1:
                    return False
    return True

# Algoritmo de backtracking
def backtracking(asignacion, examenes_disponibles):
    if len(asignacion) == len(examenes):
        return asignacion

    examen = examenes_disponibles[0]
    for dia in dias:
        asignacion[examen] = dia
        if es_valida(asignacion):
            resultado = backtracking(asignacion.copy(), examenes_disponibles[1:])
            if resultado:
                return resultado
        del asignacion[examen]
    return None

# Ejecutar y medir tiempo
inicio = time.time()
solucion = backtracking({}, examenes)
fin = time.time()
tiempo_total = fin - inicio

# Mostrar resultados
print("Solución encontrada con Backtracking:")
for examen, dia in solucion.items():
    print(f"{examen} -> {dia}")
print(f"\nTiempo de ejecución: {tiempo_total:.6f} segundos")


Solución encontrada con Backtracking:
E1 -> Lunes
E2 -> Martes
E3 -> Miércoles
E4 -> Miércoles
E5 -> Lunes
E6 -> Martes
E7 -> Martes

Tiempo de ejecución: 0.000234 segundos


## Algoritmo Beam Search

In [10]:
import time
import copy

examenes = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7']
dias = ['Lunes', 'Martes', 'Miércoles']
estudiantes = {
    'A': ['E1', 'E2', 'E3'],
    'B': ['E2', 'E4', 'E5'],
    'C': ['E1', 'E4', 'E6'],
    'D': ['E3', 'E5', 'E7']
}

def es_valida(asignacion):
    examenes_por_dia = {dia: [] for dia in dias}
    for examen, dia in asignacion.items():
        examenes_por_dia[dia].append(examen)

    for estudiante, cursos in estudiantes.items():
        examenes_estudiante = {dia: 0 for dia in dias}
        for curso in cursos:
            if curso in asignacion:
                dia = asignacion[curso]
                examenes_estudiante[dia] += 1
                if examenes_estudiante[dia] > 1:
                    return False
    return True

def calcular_peso(asignacion):
    return sum(1 for e in examenes if e in asignacion)

def beam_search(k):
    candidatos = [{}]  
    for examen in examenes:
        nuevos_candidatos = []
        for asignacion in candidatos:
            for dia in dias:
                nueva_asignacion = copy.deepcopy(asignacion)
                nueva_asignacion[examen] = dia
                if es_valida(nueva_asignacion):
                    nuevos_candidatos.append(nueva_asignacion)
        nuevos_candidatos.sort(key=calcular_peso, reverse=True)
        candidatos = nuevos_candidatos[:k]
        if not candidatos:
            break
    return candidatos[0] if candidatos else None

inicio = time.time()
solucion_beam = beam_search(k=3)  
fin = time.time()
tiempo_beam = fin - inicio

print("Solución encontrada con Beam Search:")
if solucion_beam:
    for examen, dia in solucion_beam.items():
        print(str(examen) +" -> " +str(dia))
else:
    print("No se encontró solución.")
print("\nTiempo de ejecución: "+str(tiempo_beam) +" segundos")


Solución encontrada con Beam Search:
E1 -> Lunes
E2 -> Martes
E3 -> Miércoles
E4 -> Miércoles
E5 -> Lunes
E6 -> Martes
E7 -> Martes

Tiempo de ejecución: 0.0 segundos


## Algoritmo Local Search

In [3]:
import time
import random
import copy

examenes = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7']
dias = ['Lunes', 'Martes', 'Miércoles']
estudiantes = {
    'A': ['E1', 'E2', 'E3'],
    'B': ['E2', 'E4', 'E5'],
    'C': ['E1', 'E4', 'E6'],
    'D': ['E3', 'E5', 'E7']
}

def es_valida(asignacion):
    for estudiante, cursos in estudiantes.items():
        examenes_por_dia = {dia: 0 for dia in dias}
        for curso in cursos:
            if curso in asignacion:
                dia = asignacion[curso]
                examenes_por_dia[dia] += 1
                if examenes_por_dia[dia] > 1:
                    return False
    return True

def contar_violaciones(asignacion):
    violaciones = 0
    for estudiante, cursos in estudiantes.items():
        examenes_por_dia = {dia: 0 for dia in dias}
        for curso in cursos:
            if curso in asignacion:
                dia = asignacion[curso]
                examenes_por_dia[dia] += 1
        for cuenta in examenes_por_dia.values():
            if cuenta > 1:
                violaciones += (cuenta - 1)
    return violaciones

def local_search(max_iter=100):
    asignacion = {examen: random.choice(dias) for examen in examenes}

    for _ in range(max_iter):
        mejorado = False
        for examen in examenes:
            mejor_dia = asignacion[examen]
            menor_violacion = contar_violaciones(asignacion)

            for dia in dias:
                nueva = asignacion.copy()
                nueva[examen] = dia
                v = contar_violaciones(nueva)
                if v < menor_violacion:
                    mejor_dia = dia
                    menor_violacion = v
                    mejorado = True

            asignacion[examen] = mejor_dia

        if not mejorado:
            break  

    return asignacion

inicio = time.time()
solucion_local = local_search(max_iter=200)
fin = time.time()
tiempo_local = fin - inicio

print("Solución encontrada con Local Search:")
for examen, dia in solucion_local.items():
    print(f"{examen} -> {dia}")

print(f"\nAsignación válida: {'Sí' if es_valida(solucion_local) else 'No'}")
print(f"Violaciones: {contar_violaciones(solucion_local)}")
print(f"Tiempo de ejecución: {tiempo_local * 1000:.10f} ms")

Solución encontrada con Local Search:
E1 -> Miércoles
E2 -> Martes
E3 -> Martes
E4 -> Lunes
E5 -> Miércoles
E6 -> Martes
E7 -> Lunes

Asignación válida: No
Violaciones: 1
Tiempo de ejecución: 0.0000000000 ms



### Tabla comparativa de resultados
| Algoritmo         | ¿Solución válida? | Violaciones | Tiempo de ejecución |
|-------------------|-------------------|-------------|---------------------|
| Backtracking      | Sí                | 0           | 1.033 ms            |
| Beam Search (k=3) | Sí                | 0           | ~0.01 ms            |
| Local Search      | No                | 1           | 0.9954 ms           |


### Discusión de Resultados
En nuestras pruebas, el algoritmo de Backtracking fue capaz de encontrar una solución completamente válida, aunque su tiempo de ejecución fue ligeramente mayor comparado con los otros métodos. Esto es esperado, ya que backtracking explora muchas combinaciones posibles para garantizar una solución correcta.

Por otro lado, Beam Search no solo encontró una solución válida, sino que lo hizo en el menor tiempo. Esto demuestra que, con una buena elección del parámetro k, puede ser muy eficiente. En contraste, Local Search encontró una solución con una violación, lo que evidencia que no siempre garantiza validez, pero sí ofrece una ejecución bastante rápida.

### Conclusiones
Se concluir que Backtracking es el método más confiable cuando necesita una solución garantizada, pero su costo computacional puede ser alto en problemas más grandes. Beam Search se presenta como una alternativa poderosa cuando se desea rapidez con buena calidad de solución.

Finalmente, Local Search puede ser útil en situaciones donde una solución aproximada es suficiente y el tiempo es importante. En general, la elección del algoritmo dependerá del contexto del problema y del balance entre exactitud y eficiencia que se requiera.